## Linear Regression

In [22]:
import warnings

# Suppress specific warning
warnings.filterwarnings("ignore", category=YourWarningCategory, module='module_name')

# Get user input for the week number to predict
# num_week = int(input("Enter the week to predict: "))

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

positions = ["qb", "rb", "wr", "te"]
# positions = ["qb"]


for pos in positions:
    # Load your dataset
    data = pd.read_csv("datasets/weekly_scoring.csv")

    # Preprocessing
    data = data[data['POS'] == pos]
    weights = data['WEIGHT']
    
    if(pos == 'qb'):
        # data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        # 'RECEIVING LG', 'RECEIVING 20+', 'RECEIVING TD', 'RUSHING Y/A', 'RUSHING LG',
        # 'RUSHING 20+', 'DATE', 'WEIGHT', 'YEAR'])
        # Define the list of variables to predict
        var_list = ['PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 'PASSING Y/A', 'PASSING TD', 'PASSING INT',
        'PASSING SACKS', 'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'WEEK', 'AVG_FPTS', 'MAX_FPTS', 'MIN_FPTS', 'VAR_FPTS']
    if(pos == 'rb'):
        # Define the list of variables to drop and predict
        # data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 
        # 'PASSING Y/A', 'PASSING TD', 'PASSING INT', 'RECEIVING LG', 'RECEIVING 20+',
        # 'PASSING SACKS', 'YEAR', 'WEIGHT', 'DATE'])
        var_list = ['RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING TD', 'RUSHING Y/A', 'RUSHING LG',
        'RUSHING 20+', 'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'MISC FPTS', 'WEEK', 'AVG_FPTS', 'MAX_FPTS', 'MIN_FPTS', 'VAR_FPTS']
    if(pos == 'wr'):
        # Define the list of variables to drop and predict
        # data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 
        # 'PASSING Y/A', 'PASSING TD', 'PASSING INT',
        # 'PASSING SACKS', 'YEAR', 'WEIGHT', 'DATE', 'RUSHING Y/A', 'RUSHING LG', 'RUSHING 20+'])
        var_list = ['RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING TD', 'RECEIVING LG', 'RECEIVING 20+',
        'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'MISC FPTS', 'WEEK', 'AVG_FPTS', 'MAX_FPTS', 'MIN_FPTS', 'VAR_FPTS']
    if(pos == 'te'):
        # Define the list of variables to drop and predict
        # data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 
        # 'PASSING Y/A', 'PASSING TD', 'PASSING INT',
        # 'PASSING SACKS', 'YEAR', 'WEIGHT', 'DATE', 'RUSHING Y/A', 'RUSHING LG', 'RUSHING 20+'])
        var_list = ['RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING TD', 'RECEIVING LG', 'RECEIVING 20+',
        'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'MISC FPTS', 'WEEK', 'AVG_FPTS', 'MAX_FPTS', 'MIN_FPTS', 'VAR_FPTS']

# Without Player
    # # Extracting features and target variable
    # X = data[var_list]  # Ensure 'MISC FPTS' is not in var_list
    # y = data['MISC FPTS']

    # # Splitting the dataset into training and testing sets
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # # Creating and training the linear regression model
    # model = LinearRegression()
    # model.fit(X_train, y_train)

    # # Predicting on the test set
    # y_pred = model.predict(X_test)

    # # Evaluating the model (optional)
    # mse = mean_squared_error(y_test, y_pred)
    # print(f'Mean Squared Error: {mse}')

    # # Printing the coefficients
    # coefficients = pd.DataFrame({'Variable': X.columns, 'Coefficient': model.coef_})
    # print(coefficients)

# With Player
    # Extracting features and target variable (split X and Y)
    X = data[var_list]  # Ensure 'MISC FPTS' is not in var_list
    X['PLAYER'] = data['PLAYER']  # Include 'Player' as a feature

    y = data['MISC FPTS']

    # Handling missing values in numeric columns using SimpleImputer
    numeric_cols = X.select_dtypes(include='number').columns
    imputer = SimpleImputer(strategy='mean')  # You can change the strategy as needed
    X[numeric_cols] = imputer.fit_transform(X[numeric_cols])
    
    # Target encoding Player --> replaces the Player value with the mean FPTS for each player
    encoder = ce.TargetEncoder(cols=['PLAYER'])
    X = encoder.fit_transform(X, y)

    # One-hot encoding Player
    # X = pd.get_dummies(X, columns=['PLAYER'], drop_first=True)

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Creating and training the linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predicting on the test set
    y_pred = model.predict(X_test)

    # Evaluating the model (optional)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error {pos}: {mse}')

    # Printing the coefficients
    # coefficients = pd.DataFrame({'Variable': X.columns, 'Coefficient': model.coef_})
    # print(coefficients)

## Create result dataset
    import re

    data2 = pd.read_csv("datasets/weekly_scoring.csv")

    # Predicting on the entire dataset
    data['Predicted_FPTS'] = model.predict(X)

    # Organizing the results into a new DataFrame
    result_df = data[['PLAYER', 'Predicted_FPTS']].copy()

    # Grouping by 'PLAYER' and calculating the average predicted FPTS
    result_df = result_df.groupby('PLAYER').mean().reset_index()
    result_df = result_df.sort_values(by='Predicted_FPTS', ascending=False)

    pattern = r'\((.*?)\)'
    result_df['TEAM'] = result_df['PLAYER'].apply(lambda x: re.search(pattern, x).group(1) if re.search(pattern, x) else pd.NA)

    # Keep only the first unique occurrence of any value in the 'Team' column
    result_df = result_df.drop_duplicates(subset='TEAM')

    # Remove any player with the team equal to 'FA'
    result_df = result_df.query("TEAM != 'FA'")

    # Save the results to a CSV file
    file_name = f"predictions/LR{pos}.csv"
    result_df.to_csv(file_name, index=False)


/var/folders/qd/rhs7cnfj66l4p8v_yrkw8n4m0000gn/T/ipykernel_5460/3158462227.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['PLAYER'] = data['PLAYER'].copy()  # Include 'Player' as a feature
/var/folders/qd/rhs7cnfj66l4p8v_yrkw8n4m0000gn/T/ipykernel_5460/3158462227.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_cols] = imputer.fit_transform(X[numeric_cols])
/var/folders/qd/rhs7cnfj66l4p8v_yrkw8n4m0000gn/T/ipykernel_5460/3158462227.py:83: SettingWithCopyWarning: 
A value is trying to be

Mean Squared Error qb: 0.20118776911889436
Mean Squared Error rb: 2.043802434038341e-28


/var/folders/qd/rhs7cnfj66l4p8v_yrkw8n4m0000gn/T/ipykernel_5460/3158462227.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['PLAYER'] = data['PLAYER'].copy()  # Include 'Player' as a feature
/var/folders/qd/rhs7cnfj66l4p8v_yrkw8n4m0000gn/T/ipykernel_5460/3158462227.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_cols] = imputer.fit_transform(X[numeric_cols])
/var/folders/qd/rhs7cnfj66l4p8v_yrkw8n4m0000gn/T/ipykernel_5460/3158462227.py:83: SettingWithCopyWarning: 
A value is trying to be

Mean Squared Error wr: 2.0544864650156913e-28
Mean Squared Error te: 1.8912247444471536e-28
